# 核函数笔记
## 1 核函数
核(也称GPs中的协方差函数)是GPs的一个重要组成部分，它决定GP的先验和后验形状。它们通过定义两个数据点的“相似性”，并结合相似的数据点应该具有相似的目标值的假设，对正在学习的函数进行编码。


## 2 sklearn内置核函数
sklearn中的核函数可以使用`+`,`*`,`**`进行组合运算形成新的核函数。参考[scikit-learn高斯过程文档](https://scikit-learn.org.cn/view/86.html#1.7.5%20%E9%AB%98%E6%96%AF%E8%BF%87%E7%A8%8B%E5%86%85%E6%A0%B8)。

### 2.1 基础核
常数核函数：`ConstantKerne`，用于调整其平均值。

白噪声核函数：`WhiteKernel`，参数`noise_level`用于调整其参数水平。
### 2.2 径向基核
平方指数核：`RBF`，稳定，基本上是最常用的核函数。
### 2.3 Matérn核
马特恩核：`Matern`，是RBF核的推广，可以通过参数`nu`调整其平滑度，当$\nu  \to \infty $，该核即为RBF核。实际上，RBF核的条件过于理想，实际上在通常的学习中，利用Matern核可以允许让学习函数真正接近底层函数。
### 2.4 有理二次核
有理二次核：`RationalQuadratic`,可以看作是具有不同特征长度尺度的RBF和的混合。
### 2.5 周期核
周期核：`ExpSineSquared`，该核可以对周期函数进行建模。
### 2.6 点积核
点积核：`DotProduct`，非平稳。


## 3 如何正确选用核函数
正确选用核函数需要根据实际的数据决定，详细选取最佳的核函数参考[The Kernel Cookbook: Advice on Covariance functions](https://www.cs.toronto.edu/~duvenaud/cookbook/)。可以根据基础核运算组合成自定义核。

尽管最常用的方法是直接使用RBF作为核函数，若想要更加精确地去拟合实际的底层函数，需要对不同的核函数进行`+`运算，`*`运算进行组合，甚至自己编写相应的python函数。

如下例子（摘自[scikit-learn高斯过程文档](https://scikit-learn.org.cn/view/86.html#1.7.5%20%E9%AB%98%E6%96%AF%E8%BF%87%E7%A8%8B%E5%86%85%E6%A0%B8)），为夏威夷莫纳洛阿观测站收集的大气二氧化碳平均浓度：
> ![夏威夷莫纳洛阿观测站收集的大气二氧化碳平均浓度](https://scikit-learn.org.cn/upload/4abd53bcab7374dbe3a0b9c5065a0732.png)
此示例基于[RW2006]第5.4.3节。给出了一个基于对数边际似然梯度上升的复杂核工程和超参数优化的实例。这些数据包括1958年至1997年期间在夏威夷莫纳洛阿观测站收集的大气二氧化碳平均浓度(按体积计算，以百万分之数(Ppmv)计)。目的是模拟二氧化碳浓度随时间t的变化。\
\
核由几个术语组成，它们负责解释信号的不同属性：\
\
一个长期的，平稳的上升趋势可以用`RBF`核来解释。长尺度较大的径向基函数(RBF)内核强制成平滑，没有强制趋势上升，这就留给GP选择。长度、比例尺和振幅是自由的超参数。\
\
季节性因素，由定期的 `ExpSineSquared` 内核解释，固定周期为1年。 该周期分量的长度尺度控制其平滑度是一个自由参数。 为了使准确周期性的衰减，采用带有`RBF`内核的产品。 该`RBF`组件的长度尺寸控制衰减时间，并且是另一个自由参数。\
\
较小的中期不规则性将由 `RationalQuadratic` 核来解释， `RationalQuadratic` 内核组件的长度尺度和 alpha 参数决定长度尺度的扩散性。 根据[RW2006](http://www.gaussianprocess.org/gpml/chapters/RW.pdf)，这些不规则性可以更好地由 `RationalQuadratic` 来解释， 而不是 `RBF` 核，这可能是因为它可以容纳几个长度尺度。\
\
“noise(噪声)” 一词，由一个 RBF 内核贡献组成，它将解释相关的噪声分量，如局部天气现象以及 `WhiteKernel` 对白噪声的贡献。 相对幅度和`RBF`的长度尺度是进一步的自由参数。

所得核函数为：
```python
kernel=34.4**2 * RBF(length_scale=41.8)
+ 3.27**2 * RBF(length_scale=180) * ExpSineSquared(length_scale=1.44,periodicity=1)
+ 0.446**2 * RationalQuadratic(alpha=17.7, length_scale=0.957)
+ 0.197**2 * RBF(length_scale=0.138) + WhiteKernel(noise_level=0.0336)
```


## 4 常用核函数
### 4.1 常值核函数
${\displaystyle K_{\operatorname {C} }(x,x')=C}$
### 4.2 线性核函数
${\displaystyle K_{\operatorname {L} }(x,x')=x^{T}x'}$
### 4.3 白噪声核函数
${\displaystyle K_{\operatorname {GN} }(x,x')=\sigma ^{2}\delta _{x,x'}}$
### 4.4 平方指数核函数（RBF）
${\displaystyle K_{\operatorname {SE} }(x,x')=\exp {\Big (}-{\frac {\|d\|^{2}}{2\ell ^{2}}}{\Big )}}$
### 4.5 奥恩斯坦-乌伦贝克过程
${\displaystyle K_{\operatorname {OU} }(x,x')=\exp \left(-{\frac {|d|}{\ell }}\right)}$
### 4.6 马特恩核函数
${\displaystyle K_{\operatorname {Matern} }(x,x')={\frac {2^{1-\nu }}{\Gamma (\nu )}}{\Big (}{\frac {{\sqrt {2\nu }}|d|}{\ell }}{\Big )}^{\nu }K_{\nu }{\Big (}{\frac {{\sqrt {2\nu }}|d|}{\ell }}{\Big )}}$
### 4.7 周期核函数
${\displaystyle K_{\operatorname {P} }(x,x')=\exp \left(-{\frac {2\sin ^{2}\left({\frac {d}{2}}\right)}{\ell ^{2}}}\right)}$
### 4.8 有理数平方核函数
${\displaystyle K_{\operatorname {RQ} }(x,x')=(1+|d|^{2})^{-\alpha },\quad \alpha \geq 0}$